In [1]:
from Bio.Entrez.Parser import StringElement
from Bio import Entrez
import pandas as pd
from typing import List, Dict, Union
from tqdm import tqdm
import time

Entrez.email = "Your.Name.Here@example.org"

In [2]:
def bacteria_species_to_id(bac_name: str) -> Dict[str, Union[str, None]]:
    """
    It takes a bacteria name as a string, searches the NCBI taxonomy database for the bacteria name, and returns the
    bacteria's taxonomy ID as a string

    :param bac_name: the name of the bacteria species you want to find the ID for
    :type bac_name: str
    :return: A dictionary with the bacteria name as the key and the taxonomy ID as the value.
    """
    handle = Entrez.esearch(db="taxonomy", retmax=1, term=bac_name)
    records = Entrez.read(handle)
    handle.close()
    return {bac_name: records['IdList'][0]}


def fetch_bacteria_lineage(bac_id: str) -> StringElement:
    """
    It takes a bacteria ID and returns the lineage of that bacteria

    :param bac_id: The ID of the bacteria you want to get the lineage of
    :type bac_id: str
    :return: A list of dictionaries.
    """
    handle = Entrez.efetch(db="taxonomy", id=bac_id)
    record = Entrez.read(handle)
    handle.close()

    return record[0]["LineageEx"]


def id_list_to_lineage(id_list: List[Dict]) -> Dict:
    """
    > This function takes a list of dictionaries, where each dictionary contains a bacteria name and a bacteria ID, and
    returns a dictionary where each bacteria name is mapped to a list of its lineage

    :param id_list: a list of dictionaries, where each dictionary has a bacteria name as the key and a bacteria ID as the
    value
    :type id_list: StringElement
    :return: A dictionary with the bacteria name as the key and the lineage as the value.
    """
    lineage_dict = {}
    for bacteria_dict in tqdm(id_list):
        for bac_name, bac_id in bacteria_dict.items():
            lineage_dict[bac_name] = fetch_bacteria_lineage(bac_id)
            time.sleep(1)
    return lineage_dict


def lineage_to_df(lineage_dict: Dict) -> pd.DataFrame:
    """
    It takes a dictionary of dictionaries and turns it into a dataframe

    :param lineage_dict: A dictionary of lineages, where the keys are the species names and the values are the lineages
    :type lineage_dict: Dict
    :return: A dataframe with the bacteria species as the index and the columns are the taxonomic levels.
    """
    df_dict = {}
    for bacteria_species, bacteria_lineage in lineage_dict.items():
        tmp = pd.DataFrame(lineage_dict[bacteria_species])
        tmp['species'] = [bacteria_species] * tmp.shape[0]
        df_dict[bacteria_species] = tmp

    return pd.concat(df_dict, axis=1).stack().T

In [3]:
df = pd.read_csv('../Input/Bacteria_encoding.csv')
df.replace({'Pseudomona aeruginosa': 'Pseudomonas aeruginosa'}, inplace=True)
df.head()

,Organism species (sth),MRNs
0,Actinobacillus actinomycetemcomitans,AB
1,Bacillus subtilis,BS
2,Clostridium acetobutylicum,CA
3,Campylobacter jejuni,CJ
4,Chlamydia pneumoniae,CQ


In [4]:
id_list = [bacteria_species_to_id(bacteria_species) for bacteria_species in tqdm(df.iloc[:, 0].tolist())]

100%|██████████| 26/26 [00:25<00:00,  1.00it/s]


In [5]:
lineage_dict = id_list_to_lineage(id_list)

100%|██████████| 26/26 [00:57<00:00,  2.23s/it]


In [6]:
tax_df = lineage_to_df(lineage_dict)

In [7]:
tax_df

0                                \
                                       TaxId      ScientificName     Rank   
Actinobacillus actinomycetemcomitans  131567  cellular organisms  no rank   
Bacillus subtilis                     131567  cellular organisms  no rank   
Clostridium acetobutylicum            131567  cellular organisms  no rank   
Campylobacter jejuni                  131567  cellular organisms  no rank   
Chlamydia pneumoniae                  131567  cellular organisms  no rank   
Chlamydia trachomatis                 131567  cellular organisms  no rank   
Deinococcus radiodurans               131567  cellular organisms  no rank   
Escherichia coli                      131567  cellular organisms  no rank   
Enterococcus faecalis                 131567  cellular organisms  no rank   
Haemophilus influenzae                131567  cellular organisms  no rank   
Helicobacter pylori                   131567  cellular organisms  no rank   
Mycobacterium bovis                   131567  cellular organisms  no rank   
Mycoplasma genitalium                 131567  cellular organisms  no rank   
Mycobacterium leprae                  131567  cellular organisms  no rank   
Mycoplasma pneumoniae                 131567  cellular organisms  no rank   
Mycobacterium tuberculosis            131567  cellular organisms  no rank   
Neisseria gonorrhoeae                 131567  cellular organisms  no rank   
Neisseria meningitidis                131567  cellular organisms  no rank   
Pseudomonas aeruginosa                131567  cellular organisms  no rank   
Porphyromonas gingivalis              131567  cellular organisms  no rank   
Streptococcus pneumoniae              131567  cellular organisms  no rank   
Rhodobacter capsulatus                131567  cellular organisms  no rank   
Saccharomyces cerevisiae              131567  cellular organisms  no rank   
Streptococcus pyogenes                131567  cellular organisms  no rank   
Salmonella typhi                      131567  cellular organisms  no rank   
Yersinia pestis                       131567  cellular organisms  no rank   

                                                                            \
                                                                   species   
Actinobacillus actinomycetemcomitans  Actinobacillus actinomycetemcomitans   
Bacillus subtilis                                        Bacillus subtilis   
Clostridium acetobutylicum                      Clostridium acetobutylicum   
Campylobacter jejuni                                  Campylobacter jejuni   
Chlamydia pneumoniae                                  Chlamydia pneumoniae   
Chlamydia trachomatis                                Chlamydia trachomatis   
Deinococcus radiodurans                            Deinococcus radiodurans   
Escherichia coli                                          Escherichia coli   
Enterococcus faecalis                                Enterococcus faecalis   
Haemophilus influenzae                              Haemophilus influenzae   
Helicobacter pylori                                    Helicobacter pylori   
Mycobacterium bovis                                    Mycobacterium bovis   
Mycoplasma genitalium                                Mycoplasma genitalium   
Mycobacterium leprae                                  Mycobacterium leprae   
Mycoplasma pneumoniae                                Mycoplasma pneumoniae   
Mycobacterium tuberculosis                      Mycobacterium tuberculosis   
Neisseria gonorrhoeae                                Neisseria gonorrhoeae   
Neisseria meningitidis                              Neisseria meningitidis   
Pseudomonas aeruginosa                              Pseudomonas aeruginosa   
Porphyromonas gingivalis                          Porphyromonas gingivalis   
Streptococcus pneumoniae                          Streptococcus pneumoniae   
Rhodobacter capsulatus                              Rhodobacter capsulatus   
Saccharomyces

In [8]:
tax_df.to_csv('../Output/taxonomy_df.csv')